In [1]:
# Standard data science libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
import copy as cp
import time

plt.close('all')
sns.set(color_codes=True)

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize, least_squares

from sklearn.linear_model import Lasso, LinearRegression

import scipy.stats as st
from scipy import signal
import scipy
from scipy.interpolate import interp1d
from scipy.fftpack import fft, ifft, rfft, irfft

import statsmodels.api as sm

from collections import Counter

import itertools
from itertools import permutations

import sys

import pyunicorn
from pyunicorn import timeseries
from pyunicorn.timeseries.surrogates import Surrogates

from matplotlib import cm

import os
from os import listdir
from os.path import isfile, join

import random

import ray

import NeuronVasomotionFunc as nvf

import pickle

%matplotlib inline

pyunicorn: Package netCDF4 could not be loaded. Some functionality in class Data might not be available!
pyunicorn: Package netCDF4 could not be loaded. Some functionality in class NetCDFDictionary might not be available!


In [2]:
@ray.remote
def Coincidence_Distr(num_iter, des_num_neurons, rand_seed, PO2_ch, beg_ind, end_ind, alpha, 
                      lmbda, Tw, w, ma_w, max_tau, N, lasso, tl_alpha, smooth_files,
                      num_files, create_simulation, randomize, LFP_ind):    
    
    # Initialize Random Seed
    np.random.seed(rand_seed)
    
    print(num_iter)
    
    LFP_identifier = LFP_ind
    
    # Create list of lags
    tau_list = np.arange(max_tau+1)
    
    bs_sig_exp = np.zeros((num_iter,1))
    bs_score_mat = np.empty((num_iter, num_files, (end_ind-beg_ind) - Tw))
    bs_neuron_data_storage = np.empty((num_iter, num_files, Tw, des_num_neurons))
    bs_PO2_data_storage = np.empty((num_iter, num_files, Tw, 1))
    bs_corr_mat = np.zeros((num_iter, num_files, des_num_neurons))
    bs_pvalue_mat = np.zeros((num_iter, num_files, des_num_neurons))

    exp_list = []
    sig_LFP_exp = []
    sig_exp_list = []
    
    for z in np.arange(num_iter):

        sig_exp_counter = 0

        print(z)

        for i in np.arange(num_files):

            print(i)

            if randomize:
                # Uniformly select one of the exoeriments from the sample
                smooth_ind = np.random.randint(num_files)
            else:
                smooth_ind = i

            # Pull the smooth data files
            temp_smooth_file = smooth_files[smooth_ind]
            temp_smooth_path = join(smooth_path, temp_smooth_file)
            temp_data_smooth = np.genfromtxt(temp_smooth_path, delimiter = ',')
            
            exp_list.append(temp_smooth_file)

            # Organize data into single unit data and PO2 data
            if LFP_ind == None:
                X = temp_data_smooth[beg_ind:end_ind, 2:]
                
                num_neurons = X.shape[1]
            else:
                X = temp_data_smooth[beg_ind:end_ind, 2+LFP_ind]

                num_neurons = 1

            if create_simulation:
                if num_neurons < des_num_neurons:
                    X = np.concatenate((X, X[:,0:(des_num_neurons-num_neurons)]), axis=1)
                elif num_neurons > des_num_neurons:
                    # Truncate neuron data have desired number (using random uniform sample)
                    rand_inds_neur = random.sample(list(np.arange(num_neurons)), des_num_neurons)
                    X = X[:, rand_inds_neur]
                    X_raw = X_raw[:, rand_inds_neur]
                
                if des_num_neurons == 1:
                    num_neurons = 1
                else:    
                    num_neurons = X.shape[1]
            
            T = temp_data_smooth[beg_ind:end_ind, 0]
            L = temp_data_smooth[beg_ind:end_ind, PO2_ch]

            # Standardize the data
            X_strd = (X - np.mean(X, axis = 0))/np.std(X, axis = 0)
            L_strd = (L - np.mean(L))/np.std(L)

            # Determine number of neurons
            m = num_neurons

            if create_simulation:
                ## Creat Surrogate Data
                X_surr, L_surr = nvf.create_Surr_data(X, L_strd)

                # Create rolling time windows
                time_window_mat_X_orig, t_vec = nvf.createTimeWindows(X_surr, Tw, w)
                time_window_mat_L_orig, _ = nvf.createTimeWindows(L_surr, Tw, w)
            else:
                time_window_mat_X_orig, t_vec = nvf.createTimeWindows(X, Tw, w)
                time_window_mat_L_orig, _ = nvf.createTimeWindows(L, Tw, w) 
                
            num_time_windows = time_window_mat_X_orig.shape[2]

            score_vec = np.zeros(num_time_windows)
            coef_mat = np.zeros((m, num_time_windows))

            time_lag_corr_mat = np.zeros((len(tau_list), m, num_time_windows));
            sig_tl_corr_mat = np.zeros((m,num_time_windows))

            for t in np.arange(num_time_windows):

                data_tw_filt_X_orig = time_window_mat_X_orig[:,:,t]
                data_tw_filt_L_orig = time_window_mat_L_orig[:,:,t]

                if np.isnan(data_tw_filt_L_orig).any():
                    break

                if np.isinf(data_tw_filt_L_orig).any():
                    break

                if np.isnan(data_tw_filt_X_orig).any():
                    break

                if np.isinf(data_tw_filt_X_orig).any():
                    break

                X_norm_temp_orig = (data_tw_filt_X_orig - np.mean(data_tw_filt_X_orig, axis = 0))/np.std(data_tw_filt_X_orig, axis = 0)
                L_norm_temp_orig = (data_tw_filt_L_orig - np.mean(data_tw_filt_L_orig, axis = 0))/np.std(data_tw_filt_L_orig, axis = 0)

                max_lag = np.max(tau_list);

                tau_shifted_L = L_norm_temp_orig[max_lag:]
                sig_lag_X_matrix = np.zeros((Tw-max_lag, m));

                for n in np.arange(m):
                    X_series = X_norm_temp_orig[:,n];
                    temp_corrs, temp_lag_X_matrix = nvf.detCorrSigTimeLags(X_series, L_norm_temp_orig.ravel(), tau_list);

                    time_lag_corr_mat[:,n,t] = temp_corrs;
                    sig_corr_ind = np.argmax(temp_corrs);
                    sig_tl_corr_mat[n,t] = temp_corrs[sig_corr_ind]
                    sig_lag_X_matrix[:,n] = temp_lag_X_matrix[:,sig_corr_ind]

                if np.isnan(X_norm_temp_orig).any():
                    break

                if np.isinf(X_norm_temp_orig).any():
                    break   

                const_vec = np.ones((sig_lag_X_matrix.shape[0],1))

                indep_X = np.concatenate((const_vec, sig_lag_X_matrix), axis = 1)

                if lasso:
                    temp_clf = linear_model.Lasso(alpha=lmbda)
                    temp_clf.fit(indep_X, tau_shifted_L)

                    temp_score = temp_clf.score(indep_X, tau_shifted_L)
                    temp_coef = temp_clf.coef_
                else:
                    model = sm.OLS(tau_shifted_L, indep_X, axis=1)
                    res = model.fit()

                    temp_score = res.rsquared
                    temp_coef = res.params

                score_vec[t] = np.power(temp_score,1/2)
                coef_mat[:,t] = temp_coef[1:]

            adj_time_vec = (t_vec-Tw/2)/(2*60)

            max_mid_window = np.argmax(score_vec)
            
            target_corrs = sig_tl_corr_mat[:,max_mid_window]
            
            rec_time_window = [max_mid_window/2, (max_mid_window + Tw)/2]

            data_tw_filt_X_orig = time_window_mat_X_orig[:,:,max_mid_window]
            data_tw_filt_L_orig = time_window_mat_L_orig[:,:,max_mid_window]

            X_norm_temp_orig = (data_tw_filt_X_orig - np.mean(data_tw_filt_X_orig, axis = 0))/np.std(data_tw_filt_X_orig, axis = 0)
            L_norm_temp_orig = (data_tw_filt_L_orig - np.mean(data_tw_filt_L_orig, axis = 0))/np.std(data_tw_filt_L_orig, axis = 0)

            corr_pvalues, null_corr_values = nvf.det_pvalue(X_norm_temp_orig, L_norm_temp_orig, target_corrs,
                                                        tau_list, 1000)

            corr_pvalues = corr_pvalues.ravel()
            
            actual_num_var = len(target_corrs)
            
            bs_score_mat[z,i,:] = score_vec
            bs_corr_mat[z,i,:actual_num_var] = target_corrs
            bs_pvalue_mat[z,i,:actual_num_var] = corr_pvalues

            bs_PO2_data_storage[z, i, :, :] = data_tw_filt_L_orig
            bs_neuron_data_storage[z, i, :, :actual_num_var] = data_tw_filt_X_orig
            
            
            if LFP_ind == None:
                bonf_alpha = alpha/actual_num_var
            else:
                bonf_alpha = alpha/7
            
            temp_count = np.sum(corr_pvalues < bonf_alpha)
            
            if temp_count > 0:
                print("Significant experiment")
                sig_exp_counter += 1
                
                sig_exp_list.append(temp_smooth_file)

        print(sig_exp_counter)
        bs_sig_exp[z] = sig_exp_counter

    return [bs_sig_exp, bs_score_mat, bs_neuron_data_storage, bs_PO2_data_storage, bs_corr_mat, bs_pvalue_mat, exp_list, LFP_identifier, sig_exp_list]

In [3]:
@ray.remote
def Coincidence_Distr_Cholesky(num_iter, des_num_neurons, rand_seed, PO2_ch, beg_ind, end_ind, alpha, 
                      lmbda, Tw, w, ma_w, max_tau, N, lasso, tl_alpha, smooth_files,
                      num_files, create_simulation, randomize, LFP_ind, store_synch):    
    
    # Initialize Random Seed
    np.random.seed(rand_seed)
    
    #print(num_iter)
    
    LFP_identifier = LFP_ind
    
    # Create list of lags
    tau_list = np.arange(max_tau+1)
    
    #Initialize core outputs
    bs_sig_exp = np.zeros((num_iter,1))
    bs_score_mat = np.empty((num_iter, num_files, (end_ind-beg_ind) - Tw))
    bs_neuron_data_storage = np.empty((num_iter, num_files, Tw, des_num_neurons))
    bs_PO2_data_storage = np.empty((num_iter, num_files, Tw, 1))
    bs_corr_mat = np.zeros((num_iter, num_files, des_num_neurons))
    bs_pvalue_mat = np.zeros((num_iter, num_files, des_num_neurons))
    
    #Initialize optional outputs
    bs_SI_mat = np.empty((num_iter, num_files, (end_ind-beg_ind) - Tw))

    exp_list = []
    sig_LFP_exp = []
    sig_exp_list = []
    
    for z in tqdm(np.arange(num_iter)):

        sig_exp_counter = 0

        for i in tqdm(np.arange(num_files), leave=False):

            if randomize:
                # Uniformly select one of the exoeriments from the sample
                smooth_ind = np.random.randint(num_files)
            else:
                smooth_ind = i

            # Pull the smooth data files
            temp_smooth_file = smooth_files[smooth_ind]
            temp_smooth_path = join(smooth_path, temp_smooth_file)
            temp_data_smooth = np.genfromtxt(temp_smooth_path, delimiter = ',')
            
            exp_list.append(temp_smooth_file)

            # Organize data into single unit data and PO2 data
            if LFP_ind == None:
                X = temp_data_smooth[beg_ind:end_ind, 2:]
                
                num_neurons = X.shape[1]
            else:
                X = temp_data_smooth[beg_ind:end_ind, 2+LFP_ind]

                num_neurons = 1

            if create_simulation:
                if num_neurons < des_num_neurons:
                    #X = np.concatenate((X, X[:,0:(des_num_neurons-num_neurons)]), axis=1)
                    continue
                elif num_neurons > des_num_neurons:
                    # Truncate neuron data have desired number (using random uniform sample)
                    rand_inds_neur = random.sample(list(np.arange(num_neurons)), des_num_neurons)
                    X = X[:, rand_inds_neur]
                
                if des_num_neurons == 1:
                    num_neurons = 1
                else:    
                    num_neurons = X.shape[1]
            
            T = temp_data_smooth[beg_ind:end_ind, 0]
            L = temp_data_smooth[beg_ind:end_ind, PO2_ch]

            # Standardize the data
            X_strd = (X - np.mean(X, axis = 0))/np.std(X, axis = 0)
            L_strd = (L - np.mean(L))/np.std(L)

            # Determine number of neurons
            m = num_neurons

            if create_simulation:
                ## Creat Surrogate Data
                X_surr, L_surr = nvf.create_Surr_data(X, L_strd)

                # Create rolling time windows
                time_window_mat_X_orig, t_vec = nvf.createTimeWindows(X_surr, Tw, w)
                time_window_mat_L_orig, _ = nvf.createTimeWindows(L_surr, Tw, w)
            else:
                time_window_mat_X_orig, t_vec = nvf.createTimeWindows(X, Tw, w)
                time_window_mat_L_orig, _ = nvf.createTimeWindows(L, Tw, w)  
                
            num_time_windows = time_window_mat_X_orig.shape[2]

            score_vec = np.zeros(num_time_windows)
            coef_mat = np.zeros((m, num_time_windows))

            time_lag_corr_mat = np.zeros((len(tau_list), m, num_time_windows));
            sig_tl_corr_mat = np.zeros((m,num_time_windows))

            for t in np.arange(num_time_windows):

                data_tw_filt_X_orig = time_window_mat_X_orig[:,:,t]
                data_tw_filt_L_orig = time_window_mat_L_orig[:,:,t]
                
                if np.isnan(data_tw_filt_L_orig).any() or np.isinf(data_tw_filt_L_orig).any():
                    print('PO2_break')
                    break

                if np.isnan(data_tw_filt_X_orig).any() or np.isinf(data_tw_filt_X_orig).any():
                    print('SU_break')
                    break

                X_norm_temp_orig = (data_tw_filt_X_orig - np.mean(data_tw_filt_X_orig, axis = 0))/np.std(data_tw_filt_X_orig, axis = 0)
                L_norm_temp_orig = (data_tw_filt_L_orig - np.mean(data_tw_filt_L_orig, axis = 0))/np.std(data_tw_filt_L_orig, axis = 0)

                if create_simulation:
                    rand_tw_beg_ind = t
                    rand_tw_end_ind = rand_tw_beg_ind + Tw
                    
                    actual_X = X_strd[rand_tw_beg_ind:rand_tw_end_ind, :]
                    
                    valid_indices = []
                    
                    num_zero_column_add = 0
                    
                    for temp_i in np.arange(X_norm_temp_orig.shape[1]):
                        temp_var1 = np.var(X_norm_temp_orig[:,temp_i])
                        temp_var2 = np.var(actual_X[:,temp_i])
                        
                        if temp_var1 > 0 and temp_var2 > 0:
                            valid_indices.append(temp_i)
                        else:
                            num_zero_column_add += 1
                    
                    X_norm_temp_orig = X_norm_temp_orig[:,valid_indices]
                    actual_X = actual_X[:,valid_indices]
                    
                    if num_zero_column_add > 0.1:
                        filler_mat = np.zeros((Tw, num_zero_column_add))
                        time_window_mat_X_orig[:,:,t] = np.concatenate((X_norm_temp_orig, filler_mat), axis=1)
                    else:
                        time_window_mat_X_orig[:,:,t] = X_norm_temp_orig
                    
                    surr_cov_X = np.cov(X_norm_temp_orig.T)
                    actual_cov_X = np.cov(actual_X.T)
                    
                    #if not np.all(np.linalg.eigvals(surr_cov_X) > 0):
                        #print(np.linalg.eigvals(surr_cov_X))
                        
                    #    for temp_i in np.arange(X_norm_temp_orig.shape[1]):
                    #        print(np.var(X_norm_temp_orig[:,temp_i]))

                    chol_L_reverse = np.linalg.cholesky(surr_cov_X)
                    
                    X_norm_temp_orig_decorr = np.matmul(np.linalg.inv(chol_L_reverse), X_norm_temp_orig.T)
                    X_norm_temp_orig_decorr = X_norm_temp_orig_decorr.T
                    
                    #if not np.all(np.linalg.eigvals(actual_cov_X) > 0):
                        #print(np.linalg.eigvals(actual_cov_X))
                        
                    #    for temp_i in np.arange(actual_X.shape[1]):
                    #        print(np.var(actual_X[:,temp_i]))
                    
                    chol_L_actual = np.linalg.cholesky(actual_cov_X)
                    
                    X_norm_temp_orig = np.matmul(chol_L_actual, X_norm_temp_orig_decorr.T)
                    X_norm_temp_orig = X_norm_temp_orig.T
                    
                m = X_norm_temp_orig.shape[1]
                
                tau_shifted_L = L_norm_temp_orig[max_tau:]
                sig_lag_X_matrix = np.zeros((Tw-max_tau, m));
                
                for n in np.arange(m):
                    X_series = X_norm_temp_orig[:,n];
                    temp_corrs, temp_lag_X_matrix = nvf.detCorrSigTimeLags(X_series, L_norm_temp_orig.ravel(), tau_list);

                    time_lag_corr_mat[:,n,t] = temp_corrs;
                    sig_corr_ind = np.argmax(temp_corrs);
                    sig_tl_corr_mat[n,t] = temp_corrs[sig_corr_ind]
                    sig_lag_X_matrix[:,n] = temp_lag_X_matrix[:,sig_corr_ind]

                if np.isnan(X_norm_temp_orig).any():
                    break

                if np.isinf(X_norm_temp_orig).any():
                    break   

                const_vec = np.ones((sig_lag_X_matrix.shape[0],1))

                indep_X = np.concatenate((const_vec, sig_lag_X_matrix), axis = 1)

                if lasso:
                    temp_clf = linear_model.Lasso(alpha=lmbda)
                    temp_clf.fit(indep_X, tau_shifted_L)

                    temp_score = temp_clf.score(indep_X, tau_shifted_L)
                    temp_coef = temp_clf.coef_
                else:
                    model = sm.OLS(tau_shifted_L, indep_X, axis=1)
                    res = model.fit()

                    temp_score = res.rsquared
                    temp_coef = res.params

                score_vec[t] = np.power(temp_score,1/2)
                
                temp_coef = temp_coef[1:]
                
                if len(temp_coef) < coef_mat.shape[0]:
                    coef_mat[:,t] = np.concatenate((temp_coef, np.zeros(coef_mat.shape[0]-len(temp_coef))))
                else:
                    coef_mat[:,t] = temp_coef

                if store_synch:
                    X_p = X_norm_temp_orig.T

                    cov_mat = np.cov(X_p)

                    eig_values, eig_vecs = np.linalg.eig(cov_mat)

                    sort_ind = np.argsort(eig_values)
                    sorted_eig_values = eig_values[sort_ind]
                    sorted_eig_vecs = eig_vecs[:,sort_ind]

                    pca = X_p

                    smooth = [1,6]
                    
                    temp_bonf_alpha = alpha/m

                    sync_ind, _, _, _ = nvf.CMA_Sync(X_norm_temp_orig, N, sorted_eig_values, 
                                                 sorted_eig_vecs, temp_bonf_alpha, smooth)

                    sync_ind1_value = sync_ind[-1]
                    
                    bs_SI_mat[z,i,t] = sync_ind1_value
                    
            adj_time_vec = (t_vec-Tw/2)/(2*60)

            max_mid_window = np.argmax(score_vec)
            
            target_corrs = sig_tl_corr_mat[:,max_mid_window]
            
            rec_time_window = [max_mid_window/2, (max_mid_window + Tw)/2]

            data_tw_filt_X_orig = time_window_mat_X_orig[:,:,max_mid_window]
            data_tw_filt_L_orig = time_window_mat_L_orig[:,:,max_mid_window]

            X_norm_temp_orig = (data_tw_filt_X_orig - np.mean(data_tw_filt_X_orig, axis = 0))/np.std(data_tw_filt_X_orig, axis = 0)
            L_norm_temp_orig = (data_tw_filt_L_orig - np.mean(data_tw_filt_L_orig, axis = 0))/np.std(data_tw_filt_L_orig, axis = 0)

            corr_pvalues, null_corr_values = nvf.det_pvalue(X_norm_temp_orig, L_norm_temp_orig, target_corrs,
                                                            tau_list, 1000)

            corr_pvalues = corr_pvalues.ravel()
            
            actual_num_var = len(target_corrs)
            
            bs_score_mat[z,i,:] = score_vec
            bs_corr_mat[z,i,:actual_num_var] = target_corrs
            bs_pvalue_mat[z,i,:actual_num_var] = corr_pvalues

            bs_PO2_data_storage[z, i, :, :] = data_tw_filt_L_orig
            bs_neuron_data_storage[z, i, :, :actual_num_var] = data_tw_filt_X_orig
            
            if LFP_ind == None:
                bonf_alpha = alpha/actual_num_var
            else:
                bonf_alpha = alpha/7
            
            temp_count = np.sum(corr_pvalues < bonf_alpha)
            
            if temp_count > 0:
                print("Significant experiment")
                sig_exp_counter += 1
                
                sig_exp_list.append(temp_smooth_file)

        print("Sig exp: " + str(sig_exp_counter))
        bs_sig_exp[z] = sig_exp_counter

    return [bs_sig_exp, bs_score_mat, bs_neuron_data_storage, bs_PO2_data_storage, bs_corr_mat, bs_pvalue_mat, exp_list, LFP_identifier, sig_exp_list, bs_SI_mat]

In [ ]:
### Single Unit Data Analysis ###

create_simulation = 1      # Indicator variable for creating simulated variable. 1 = simulated data and 0 = actual data
store_synch = 1            # Indicator to store synchronization index and participation indices. 1 = store and 0 = don't store


# Set parameters depending on whether using actual or simulated data
if create_simulation == 0:
    total_num_iter = 1     # Number of iterations. For actual data this is set to 1
    num_cores = 1          # Number of cpu cores. For actual data this is set to 1
    iter_indices = [1]     # Number of iterations per core. For actual data this is set to 1
    des_num_var = 25       # Desired number of variables (SU) to run analysis on. Set to a maximum of 25.
    randomize = 0          # Indicator to randomize the selection of experiments. 1 = randomized and 0 = in order. Set to 0 for actual
else:
    total_num_iter = 100   # We iterate the total number of experiments (43), 100 times
    num_cores = 12         # Number of cpu cores for parallel processing. This machine can run a maximum of 12
    des_num_var = 15       # Maximum number of nuerons to simulate. Max set to 15.
    
    # Calculate iter_indices. This is the number of iterations run per each cpu core
    base_ind_value = np.floor(total_num_iter/num_cores)
    remainders = total_num_iter % num_cores
    iter_indices = np.ones((num_cores, 1))
    iter_indices = iter_indices*base_ind_value
    for j in np.arange(remainders):
        iter_indices[j] += 1
        
    randomize = 1         # Indicator to randomize the selection of experiments. 1 = randomized and 0 = in order. Set to 1 for actual

# Initiate the seed values for each cpu core
rand_indices = np.arange(num_cores)  

PO2_ch = 1        # PO2 channel (always set to 1, since there is only one channel)
beg_ind = 0       # Beginning time index of data (in units of .5 seconds)
end_ind = 1200    # Ending time index of data (in units of .5 seconds)
alpha = .05       # p-value level for Bonferonni correction
lmbda = .1        # Regularization parameter for lasso regression        
Tw = 240          # Size of time window for rolling window analysis          
w = Tw-1          # Overlap of time windows
ma_w = 6          # Moving average window
max_tau = 10      # Maximum time lag for determining correlations
N = 100           # Number of surrogates to generate for Synhronization index calculations
lasso = 1         # Indicator variable to indicate whether to run Lasso regression. 1 = Lasso and 0 = OLS
tl_alpha = .3     # Minimum threshold correlation threshold to record the time lag. Always set to zero.

# Initialize parallel process
ray.init(num_cpus = num_cores)

## Define folder path
smooth_path = '[Insert Folder Path]'

# Pull file names
smooth_files = [f for f in listdir(smooth_path) if isfile(join(smooth_path, f))]
smooth_files.sort()
num_files = len(smooth_files)

# Initialize output matrices
bs_sig_exp = np.zeros((total_num_iter,1))
bs_score_mat = np.empty((total_num_iter, num_files, (end_ind-beg_ind) - Tw))
bs_neuron_data_storage = np.zeros((total_num_iter, num_files, Tw, des_num_var))
bs_PO2_data_storage = np.zeros((total_num_iter, num_files, Tw, des_num_var))
bs_corr_mat = np.zeros((total_num_iter, num_files, des_num_var))
bs_pvalue_mat = np.zeros((total_num_iter, num_files, des_num_var))

start = time.time()

# Run analysis on actual/simulated code
SU_results = [Coincidence_Distr_Cholesky.remote(int(iter_indices[i]), des_num_var, int(rand_indices[i]), PO2_ch, beg_ind, end_ind, 
                                        alpha, lmbda, Tw, w, ma_w, max_tau, N, lasso, tl_alpha, smooth_files,
                                        num_files, create_simulation, randomize, None, store_synch) 
               for i in np.arange(num_cores)]

SU_results2 = ray.get(SU_results)

end = time.time()

ray.shutdown()
print(end-start)

# Store analyzed SU data in pickle format
# File names for SU: bs_SU_results_["sim" or "actual"]_detrended_[empty or "store_synch"]
# For this paper, we will use store_synch always
with open("bs_SU_results_sim_detrended_store_synch.txt", "wb") as fp:   #Pickling
    pickle.dump(SU_results2, fp)

print(len(SU_results2))

In [26]:
ray.shutdown()

In [5]:
### LFP data analysis ###

# Process similar to SU data, see comments above
 
analyze_actual_LFP = 1
analyze_simulated_LFP = 0

des_num_var = 1

PO2_ch = 1
beg_ind = 0
end_ind = 1190
alpha = .05
lmbda = .1
Tw = 240
w = Tw-1
ma_w = 6
max_tau = 10
N = 100
lasso = 1
tl_alpha = .3

bonf_alpha = alpha/7

if analyze_actual_LFP:
    total_num_iter = 1
    num_cores = 7
    
    LFP_indices = np.arange(num_cores)
    iter_indices = np.ones(num_cores)
    
    rand_indices = np.arange(num_cores)
    
    create_simulation = 0
    randomize = 0    
    
elif analyze_simulated_LFP:
    total_num_iter = 100
    num_cores = 12
    
    LFP_indices = []
    iter_indices = []
    
    for i in np.arange(10):
        LFP_indices.append(np.floor(i/2))
        iter_indices.append(50)
    
    LFP_indices.append(5)
    LFP_indices.append(6)
    
    iter_indices.append(100)
    iter_indices.append(100)
    
    rand_indices = np.arange(num_cores)
    
    create_simulation = 1
    randomize = 1
    
ray.init(num_cpus = num_cores)

## Define folder path
smooth_path = '/home/evan/Projects/NeuronVasomotion/PyFormat_Band_Data_v2'

# Pull file names
smooth_files = [f for f in listdir(smooth_path) if isfile(join(smooth_path, f))]
smooth_files.sort()
num_files = len(smooth_files)

bs_sig_exp = np.zeros((total_num_iter,1))
bs_score_mat = np.empty((total_num_iter, num_files, (end_ind-beg_ind) - Tw))
bs_neuron_data_storage = np.zeros((total_num_iter, num_files, Tw, des_num_var))
bs_PO2_data_storage = np.zeros((total_num_iter, num_files, Tw, des_num_var))
bs_corr_mat = np.zeros((total_num_iter, num_files, des_num_var))
bs_pvalue_mat = np.zeros((total_num_iter, num_files, des_num_var))

start = time.time()

LFP_results = [Coincidence_Distr.remote(int(iter_indices[i]), des_num_var, int(rand_indices[i]), PO2_ch, beg_ind, end_ind, 
                                    alpha, lmbda, Tw, w, ma_w, max_tau, N, lasso, tl_alpha, smooth_files, 
                                    num_files, create_simulation, randomize, int(LFP_indices[i])) 
           for i in np.arange(num_cores)]

LFP_results2 = ray.get(LFP_results)

end = time.time()

ray.shutdown()
print(end-start)

# Store analyzed LFP data in pickle format
# File names for LFP: bs_LFP_results_["sim" or "actual"]_detrended_[empty or "store_synch"]
with open("bs_LFP_results_actual_detrended_v6.txt", "wb") as fp:   #Pickling
    pickle.dump(LFP_results2, fp)

print(len(LFP_results2))

2024-04-03 11:27:37,177	INFO worker.py:1518 -- Started a local Ray instance.


(pid=60052) pyunicorn: Package netCDF4 could not be loaded. Some functionality in class Data might not be available!
(pid=60052) pyunicorn: Package netCDF4 could not be loaded. Some functionality in class NetCDFDictionary might not be available!
(Coincidence_Distr pid=60052) 1
(Coincidence_Distr pid=60052) 0
(Coincidence_Distr pid=60052) 0
(pid=60051) pyunicorn: Package netCDF4 could not be loaded. Some functionality in class Data might not be available!
(pid=60051) pyunicorn: Package netCDF4 could not be loaded. Some functionality in class NetCDFDictionary might not be available!
(Coincidence_Distr pid=60051) 1
(Coincidence_Distr pid=60051) 0
(Coincidence_Distr pid=60051) 0
(pid=60049) pyunicorn: Package netCDF4 could not be loaded. Some functionality in class Data might not be available!
(pid=60049) pyunicorn: Package netCDF4 could not be loaded. Some functionality in class NetCDFDictionary might not be available!
(Coincidence_Distr pid=60049) 1
(Coincidence_Distr pid=60049) 0
(Coinc

(Coincidence_Distr pid=60055) 12
(Coincidence_Distr pid=60051) 15
(Coincidence_Distr pid=60052) Significant experiment
(Coincidence_Distr pid=60052) 16
(Coincidence_Distr pid=60049) 14
(Coincidence_Distr pid=60054) 17
(Coincidence_Distr pid=60050) 15
(Coincidence_Distr pid=60055) 13
(Coincidence_Distr pid=60053) Significant experiment
(Coincidence_Distr pid=60053) 15
(Coincidence_Distr pid=60052) 17
(Coincidence_Distr pid=60054) 18
(Coincidence_Distr pid=60049) 15
(Coincidence_Distr pid=60051) 16
(Coincidence_Distr pid=60050) 16
(Coincidence_Distr pid=60055) 14
(Coincidence_Distr pid=60053) 16
(Coincidence_Distr pid=60052) 18
(Coincidence_Distr pid=60051) Significant experiment
(Coincidence_Distr pid=60051) 17
(Coincidence_Distr pid=60054) 19
(Coincidence_Distr pid=60049) Significant experiment
(Coincidence_Distr pid=60049) 16
(Coincidence_Distr pid=60050) 17
(Coincidence_Distr pid=60055) 15
(Coincidence_Distr pid=60053) 17
(Coincidence_Distr pid=60054) 20
(Coincidence_Distr pid=60052)

(Coincidence_Distr pid=60051) 36
(Coincidence_Distr pid=60050) 38
(Coincidence_Distr pid=60049) 32
(Coincidence_Distr pid=60055) Significant experiment
(Coincidence_Distr pid=60055) 33
(Coincidence_Distr pid=60054) 39
(Coincidence_Distr pid=60052) 33
(Coincidence_Distr pid=60053) Significant experiment
(Coincidence_Distr pid=60053) 37
(Coincidence_Distr pid=60051) 37
(Coincidence_Distr pid=60050) 39
(Coincidence_Distr pid=60049) Significant experiment
(Coincidence_Distr pid=60049) 33
(Coincidence_Distr pid=60052) 34
(Coincidence_Distr pid=60053) 38
(Coincidence_Distr pid=60051) 38
(Coincidence_Distr pid=60054) 40
(Coincidence_Distr pid=60050) 40
(Coincidence_Distr pid=60055) Significant experiment
(Coincidence_Distr pid=60055) 34
(Coincidence_Distr pid=60049) 34
(Coincidence_Distr pid=60053) 39
(Coincidence_Distr pid=60052) Significant experiment
(Coincidence_Distr pid=60052) 35
(Coincidence_Distr pid=60051) 39
(Coincidence_Distr pid=60054) 41
(Coincidence_Distr pid=60050) 41
(Coincide